In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import load_model
from multiprocessing import cpu_count
from warnings import catch_warnings
from warnings import filterwarnings
from pandas import read_csv
import matplotlib.pyplot as plt
from pandas import DataFrame
import time
#from datetime import datetime
import datetime
import itertools
import mysql.connector
from mysql.connector import connection
from mysql.connector import errorcode
from statistics import mean

In [ ]:
station = ['t51']
previous_id = np.zeros(len(station))

In [ ]:
def loadData(t):
    myconn = mysql.connector.connect(host = "82.180.142.51",
                                database = "u978805288_land",
                                user = "u978805288_root",
                                password= "ACSL@b123")
    (myconn)

    sql_select_query = "select sensor_id, value from data_log, info_time where data_log.info_id = info_time.info_id AND data_log.triplet_id = '"+t+"' ORDER BY data_log.info_id desc limit 60"
    date_select_query = "select Date_Time from info_time where triplet_id='"+t+"' order by info_id desc limit 1"

    cursor = myconn.cursor()
    cursor.execute(sql_select_query)
    data = cursor.fetchall()

    cursor.execute(date_select_query)
    date = cursor.fetchall()

    myconn.close()
    return data,date[0][0]


In [ ]:
triplet_id = "t51"
data, date_value = loadData(triplet_id)
print("Data:")
for row in data:
    print(row)

print("Date:", date_value)

In [ ]:
def dataPreprocess(data):
    result = []
    data.reverse()

    combined_data = []
    for i in range(0, len(data), 7):
        row = data[i:i+7]
        sensor_data = []

        for s, d in row:
            sid = s
            if sid == 's6':
                continue
            elif sid == 's5':
                x = d.split(',')[:6]
                for j in x:
                    sensor_data.append(float(j))
                x = d.split(',')[-1]
                sensor_data.append(float(x))
            elif sid.startswith('s'):
                x = d.split(',')[:]
                for j in x:
                    sensor_data.append(float(j))

        combined_data.extend(sensor_data)

    data = np.array(combined_data)
    return combined_data


preprocessed_data = dataPreprocess(data)
split_rows = [preprocessed_data[i:i+13] for i in range(0, len(preprocessed_data), 13)]

for row in split_rows:
    print(row)


[12.7, 33.0, 1054.16, 0.0, 9.17, -0.78, 0.01, -0.39, 6.65, 5.49, 3.84, 0.0, 706.0]
[11.9, 20.0, 1054.17, 0.0, 8.33, -0.78, 0.01, -0.39, 6.89, 5.37, 3.84, 0.0, 706.0]
[12.2, 29.0, 1054.25, 0.0, 8.33, -0.78, 0.0, -0.4, 6.52, 4.45, 2.38, 0.0, 705.0]
[12.1, 21.0, 1054.28, 0.0, 8.33, -0.78, 0.01, -0.4, 7.01, 5.73, 3.6, 0.0, 705.0]
[12.1, 31.0, 1054.43, 0.0, 7.5, -0.78, 0.01, -0.39, 6.52, 4.76, 3.35, 0.0, 704.0]
[9.9, 25.0, 1054.71, 0.0, 0.83, -0.77, 0.0, -0.41, 6.71, 5.49, 2.38, 0.0, 705.0]
[9.6, 26.0, 1054.8, 0.0, 0.83, -0.77, -0.0, -0.41, 6.28, 5.79, 1.34, 0.0, 704.0]
[25.5, 25.0, 1054.81, 0.0, 0.0, -0.77, 0.01, -0.42, 7.68, 5.24, 0.61, 0.0, 704.0]
[8.9, 28.0, 1054.96, 0.0, 0.0, -0.77, 0.0, -0.42, 7.07, 6.1, -0.12, 0.0, 703.0]
[8.4, 29.0, 1055.15, 0.0, 0.0, -0.77, 0.01, -0.43, 7.13, 5.49, -0.85, 0.0, 702.0]


In [ ]:
import tensorflow as tf
data_list = []
count = 0
for i in range(10):
    d = preprocessed_data[count:count+13]
    data_list.append(d)
    count += 13

data_array = np.array(data_list)
data_array = data_array.reshape((-1, 10, 13))
data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)

In [ ]:
# Load the first model
STOREDIR = ('/content/drive/MyDrive/Count_data-LMS/urni dhank/Encoder_model/finalmodel/')
model1 = tf.saved_model.load(STOREDIR)
predict_fn = model1.signatures["serving_default"]

# Load the second model
STOR = "/content/drive/MyDrive/Count_data-LMS/urni dhank/"
model2 = tf.keras.models.load_model(STOR+'Encoder_model/website1.h5')

def runModel(data_tensor):
    previous = 1
    try:
        # Run the prediction on the first model
        pred = predict_fn(data_tensor)
        pred = np.argmax(pred)
        pred = np.round(pred).astype(int)

        # Run the prediction on the second model
        dt = data_tensor[:,:,0:5]
        pred1 = model2.predict(dt)

        combined_list = []
        combined_list.extend([round(x,2) for x in pred1.tolist()[0]])
        combined_list.append(pred)

        return (combined_list)
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [ ]:
def insertDatabase(prediction, triplet_id):
    connection = mysql.connector.connect(host = "82.180.142.51",
                                database = "u978805288_land",
                                user =   "u978805288_root",
                                password= "ACSL@b123")
    try:
        (connection)
        current_time = date+ datetime.timedelta()
        print('current_time:',current_time)

        val = "(NULL,'" +(triplet_id)+"',"+ str(prediction[0])+","+ str(prediction[1])+","+str(prediction[2])+","+str(prediction[3])+","+str(prediction[4])+","+str(prediction[5])+ ",'"+str(current_time)+ "','no')"

        add_prediction = "INSERT INTO Prediction_Real_Data  VALUES "+val
        print(add_prediction)

        cursor = connection.cursor()
        cursor.execute(add_prediction)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into prediction table")
        cursor.close()

    except mysql.connector.Error as error:
        print("Failed to insert record into prediction table {}".format(error))

    finally:
        if connection.is_connected():
            connection.close()

In [ ]:
while True:
  count = 0
  for triplet_id in station:
    data,date = loadData(triplet_id)
    print(date)
    date = date+ datetime.timedelta(minutes=10)
    data = dataPreprocess(data)
    data_list = []
    count = 0
    for i in range(10):
        d = preprocessed_data[count:count+13]
        data_list.append(d)
        count += 13

    data_array = np.array(data_list)
    data_array = data_array.reshape((-1, 10, 13))
    data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
    prediction = runModel(data_tensor)
    print(prediction)
    insertDatabase(prediction,triplet_id)
    print('Enjoy')
  count+=1